In [1]:
## Tianye Song, Julina Zhang, Kerry Jones, Capstone Cyber Intrusion Detection, Formalized Code
## 3/29/17
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans


In [2]:
mon = pd.read_csv('~/Desktop/data/netFlow-02-13-2017.csv', sep = ',')
# tue = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')
# wed = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')
# thur = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')
# fri = pd.read_csv('~/Desktop/netFlow-02-11-2017.csv', sep = ',')

In [12]:
len(mon)

11277313

In [4]:
mon_collab = mon[(((mon.srcAddr == '128.143.2.75') & (mon.srcPort == 443)) | ((mon.dstAddr =='128.143.2.75') & (mon.dstPort == 443)))]

In [9]:
mon_collab.head()

,Unnamed: 0,id,srcAddr,dstAddr,srcPort,dstPort,prot,bytes,pkts,dur,rate,firstTime,lastTime,maxrate
14,14,14,128.143.2.75,98.249.10.208,443,62857,6,3000,28,0.089,269662.96,2017-02-13 14:06:27.511,2017-02-13 14:06:27.817,250966.0
20,20,20,128.143.2.75,73.99.8.4,443,50214,6,31672,52,0.181,1399867.45,2017-02-13 14:11:46.483,2017-02-13 14:11:46.824,0.0
28,28,28,184.6.45.217,128.143.2.75,55076,443,6,8684,44,0.942,73749.48,2017-02-13 14:18:47.697,2017-02-13 14:18:48.641,72390.0
43,43,43,128.143.2.75,73.152.128.219,443,50211,6,2664,28,0.073,291945.39,2017-02-13 14:12:37.704,2017-02-13 14:12:37.777,274410.0
92,92,92,73.251.173.42,128.143.2.75,50514,443,6,10172,44,0.215,378493.17,2017-02-13 14:14:02.520,2017-02-13 14:14:02.739,363013.0


In [10]:
mon_collab.tail()

,Unnamed: 0,id,srcAddr,dstAddr,srcPort,dstPort,prot,bytes,pkts,dur,rate,firstTime,lastTime,maxrate
11277230,2712376,2712376,73.99.8.12,128.143.2.75,64813,443,6,17154,60,0.158,868556.96,2017-02-14 04:42:50.683,2017-02-14 04:42:50.841,868556.0
11277237,2712383,2712383,128.143.2.75,73.201.167.179,443,57426,6,8508,42,0.090,756265.38,2017-02-14 04:54:46.490,2017-02-14 04:54:46.769,0.0
11277245,2712391,2712391,128.143.2.75,71.206.183.190,443,61664,6,5892,66,0.174,270896.51,2017-02-14 04:49:07.578,2017-02-14 04:49:07.809,242206.0
11277298,2712444,2712444,8.37.199.109,128.143.2.75,60861,443,6,16176,66,0.091,1422068.41,2017-02-14 04:34:05.542,2017-02-14 04:34:06.056,1367208.0
11277306,2712452,2712452,98.244.79.87,128.143.2.75,55453,443,6,240,6,0.000,0.00,2017-02-14 04:27:12.281,2017-02-14 04:27:12.281,0.0


In [70]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy.spatial import distance
## function adopted from paper, KSE
def euc_dis(p1,p2):
    a = p1.values
    b = p2.values
    c = np.vstack((a,b))
    dist = distance.pdist(c)
    return float(dist)
def KSE_Test(df, sample_size):
    # initiate two sample dfs, one distance matrix, and one kse score.
    # generate 2 random samples.
    sample1 = df.sample(n = sample_size, replace = False)
    sample2 = df.sample(n = sample_size, replace = False)
    kse_score = [None] * len(df) # empty list to store KSE scores
#   distance_matrix = np.matrix([[None] * int(len(df) * fraction) , 
#                               [None] * int(len(df) * fraction)])
#     distance_matrix = [[0 for x in range(len(sample1))] 
#                        for y in range(len(sample2))] 
    distance_matrix = np.zeros((sample_size, sample_size))
    # calculate distance from all points in sample1 to all points in sample2
    for i in range(0,sample_size):
        for j in range(0,sample_size):
            distance_matrix[i][j] = euc_dis(sample1.iloc[i,],sample2.iloc[j,])
    print("half way there")
    
    # calculate average KS-score for each instance in df.
    for i in range(0, len(df)):
        observed_instance = df.iloc[i,]
        distance_array = [0] * sample_size
        for j in range(0,sample_size):
            distance_array[j] = euc_dis(observed_instance,sample2.iloc[j,])
        sum = 0
        for k in range(0, sample_size):
            ks_score = stats.ks_2samp(distance_array, distance_matrix[k])[0]
            sum += ks_score
        average = sum/sample_size
        kse_score[i] = average
    return kse_score 

In [15]:
mon_collab_clean = mon_collab[["bytes","pkts","dur","rate"]]

In [21]:
# take 10% sample from population
sample_mon = mon_collab_clean.sample(frac = 0.1, replace = False)

In [66]:
len(sample_mon) * 0.01

474.78000000000003

In [71]:
kse_mon = KSE_Test(sample_mon, 10)

half way there


In [33]:
len(kse_mon)

47478

In [37]:
mon_collab_clean.iloc[0]

bytes      3000.000
pkts         28.000
dur           0.089
rate     269662.960
Name: 14, dtype: float64

In [39]:
from scipy.spatial import distance
distance.cdist(mon_collab_clean.iloc[0], mon_collab_clean.iloc[0], 'euclidean')

ValueError: XA must be a 2-dimensional array.

In [49]:
distance.cdist(mon_collab_clean.iloc[0].values, mon_collab_clean.iloc[0].values,'euclidean')

ValueError: XA must be a 2-dimensional array.

In [61]:
a = mon_collab_clean.iloc[0].values
b = mon_collab_clean.iloc[0].values
c = np.vstack((a,b))
c
dist = distance.pdist(c)
dist

array([ 0.])

In [69]:
float(euc_dis(mon_collab_clean.iloc[0],mon_collab_clean.iloc[0]))

0.0

bytes      3000.000
pkts         28.000
dur           0.089
rate     269662.960
Name: 14, dtype: float64